# Augmentation Strategies 

In this notebook we explore different augmentation strategies for neural differential equations. This time, we'll make use of the 3D `concentric spheres` dataset.

In [1]:
from prep import prep_nbook
prep_nbook()

from torchdyn.models import *; from torchdyn.data_utils import *
from torchdyn import *

SyntaxError: invalid syntax (prep.py, line 77)

**Data**

In [ ]:
import matplotlib.pyplot as plt

# Generate 3D nested spheres data
d = ToyDataset()
X, yn = d.generate(n_samples=2**13, dataset_type='spheres')   

plot_

In [ ]:
c = ['blue', 'orange']
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
for i in range(len(X)):
    ax.scatter(X[yn==i,0], X[yn==i,1], X[yn==i,2], s=5, alpha=0.5, c=c[i])

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchdiffeqplt.legend(["Class %d" % i for i in range(2)]);
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

bs = len(X)
X_train = torch.Tensor(X).to(device)
y_train = torch.LongTensor(yn.long()).to(device)
train = data.TensorDataset(X_train, y_train)
trainloader = data.DataLoader(train, batch_size=bs, shuffle=False)

**Learner**

In [ ]:
class Learner(pl.LightningModule):
    def __init__(self, model:nn.Module, settings:dict={}):
        super().__init__()
        defaults.update(settings)
        self.settings = defaults
        self.model = model
        self.c = 0
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch      
        y_hat = self.model(x)   
        loss = nn.CrossEntropyLoss()(y_hat, y)
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainloader

## Neural ODE (0--augmentation)

The first model under consideration involves 0--augmentation, and is often referred to as [ANODE](https://arxiv.org/abs/1904.01681). With `torchdyn`, turning a neural ODE into an ANODE is as simple as using the `Augmenter` class as follows:

In [ ]:
settings = {'type':'classic', 'controlled':False, 'solver':'dopri5'}

In [ ]:
func = DEFunc(nn.Sequential(nn.Linear(8,80),
                            nn.Tanh(),
                            nn.Linear(80,8))
                            )

In [ ]:
neuralDE = NeuralDE(func, settings).to(device)

model = nn.Sequential(Augmenter(augment_dims=5),
                      neuralDE,
                      nn.Linear(8,2)).to(device)

Note that `Augmenter` can be fully-specified outside the neuralDE model. This makes it straightforward to switch between augmented and non-augmented at will.

In [ ]:
learn = Learner(model, settings)
trainer = pl.Trainer(max_nb_epochs=300)
trainer.fit(learn)

## Neural ODE (Input-Layer (IL)-Augmentation) 

Input layer augmentation ([IL-augmentation](https://arxiv.org/abs/2002.08071)) is also easy to implement with `torchdyn`. The following is an example of a partial IL-augmentation where a linear layer is trained to determine the initial condition of the additional `5` dimensions. Notice how the `DEFunc` goes from `8` to `8` dimensions. 

In [ ]:
settings = {'type':'classic', 'controlled':False, 'solver':'dopri5'}

In [ ]:
func = DEFunc(nn.Sequential(nn.Linear(8,80),
                            nn.Tanh(),
                            nn.Linear(80,8))
                            )

In [ ]:
neuralDE = NeuralDE(func, settings).to(device)

model = nn.Sequential(Augmenter(augment_func=nn.Linear(3, 5)),
                      neuralDE,
                      nn.Linear(8,2)).to(device)

In [ ]:
learn = Learner(model, settings)
trainer = pl.Trainer(max_nb_epochs=300)
trainer.fit(learn)

Note that for a full `IL-augmentation`, the following model definition can replace the above. Determining the entire initial condition with a linear layer can be achieved without `Augmenter`.

In [ ]:
func = DEFunc(nn.Sequential(nn.Linear(8,80),
                            nn.Tanh(),
                            nn.Linear(80,8))
                            )

neuralDE = NeuralDE(func, settings).to(device)

model = nn.Sequential(nn.Linear(3, 8),
                      neuralDE,
                      nn.Linear(8,2)).to(device)

## Neural ODE (Higher-Order Augmentation)

Finally, we take a look at increasing the order of the ODE as a method to augment the dimensions. In `torchdyn`, the `DEFunc` can be specified to evolve according to higher orders by passing `func_type='higher_order` and `order=n` as arguments. `Augmenter` is still used to augment with `data dimension * n` additional dimensions.

In [ ]:
settings = {'type':'higher_order', 'controlled':False, 'solver':'dopri5'}

In [ ]:
func = DEFunc(nn.Sequential(nn.Linear(6,80),
                            nn.Tanh(),
                            nn.Linear(80,3)),
                            func_type='higher_order',
                            order=2)

In [ ]:
neuralDE = NeuralDE(func, settings).to(device)

model = nn.Sequential(Augmenter(augment_dims=3),
                      neuralDE,
                      nn.Linear(6,2)).to(device)

In [ ]:
learn = Learner(model, settings)
trainer = pl.Trainer(max_nb_epochs=300)
trainer.fit(learn)